In [1]:
import pandas as pd
df_imported = pd.read_csv('../../results/top_streaming_products.csv')
print(df_imported.head())



      product_name  positive_proba  \
0  Unknown Product        0.999945   
1  Unknown Product        0.999909   
2  Unknown Product        0.999908   

                                         review_text  \
0  Awesome fantastic super duper perfecto magnifi...   
1  Excellent quality. Easy to set up. Works great...   
2  Works great, highly recommend. Great alternati...   

                                          categories  
0  Back To College,College Electronics,College Tv...  
1  Back To College,College Electronics,College Tv...  
2  Back To College,College Electronics,College Tv...  


In [2]:
!pip install openai
!pip install tabulate



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
pip show openai


Name: openai
Version: 1.101.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: label-studio
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI
import os
import pandas as pd

# Load your CSV data
df_imported = pd.read_csv('../../results/top_streaming_products.csv')

# Display columns and top records for checking
print(df_imported.columns)
print(df_imported.head())

# Extract relevant product data for prompt construction using positive_proba
top_products = df_imported.sort_values('positive_proba', ascending=False).head(3)
worst_product = df_imported.sort_values('positive_proba', ascending=True).head(1)

# Format product data as text for prompt
def format_product_row(row):
    return (f"{row['product_name']} (Score: {row['positive_proba']:.2f}, "
            f"Category: {row['categories']})\nTop Review: {row['review_text']}")

top_products_text = "\n\n".join([format_product_row(row) for _, row in top_products.iterrows()])
worst_product_text = format_product_row(worst_product.iloc[0])

# Craft the system and user message for editorial quality
system_msg = "You are an expert product reviewer writing blog-style comparison articles for consumers. Use insights from The Wirecutter, The Verge, and Consumer Reviews sites."
user_msg = f"""
Write a blog-style review article based on this streaming product data:

Top 3 products:
{top_products_text}

Worst product:
{worst_product_text}

Please include:
- The top 3 products and the key differences between them
- Who should pick each product and why
- The top complaints for each product from reviews
- What is the worst product in the category and why readers should avoid it
- Reference comparison insights from trusted review sites
Your answer should be concise, insightful, and actionable.
"""

# Perplexity API details
PPLX_API_KEY = "x"
API_KEY = os.environ.get("PPLX_API_KEY", PPLX_API_KEY)
PPLX_BASE_URL = "https://api.perplexity.ai"
MODEL_NAME = "sonar-pro"

# Initialize the client
client = OpenAI(
    api_key=API_KEY,
    base_url=PPLX_BASE_URL
)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg}
]

try:
    print(f"Attempting to call {MODEL_NAME} at {PPLX_BASE_URL} via the configured SDK...")

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=0.7
    )
    print("\n--- Model Response ---")
    print(response.choices[0].message.content)
    print("----------------------")

except Exception as e:
    print(f"\nAn error occurred while calling the API: {e}")
    print("\nIMPORTANT NOTE: This environment cannot execute external API calls (like Perplexity's).")
    print("This script is now configured for your specific key/URL and must be run locally.")


Index(['product_name', 'positive_proba', 'review_text', 'categories'], dtype='object')
      product_name  positive_proba  \
0  Unknown Product        0.999945   
1  Unknown Product        0.999909   
2  Unknown Product        0.999908   

                                         review_text  \
0  Awesome fantastic super duper perfecto magnifi...   
1  Excellent quality. Easy to set up. Works great...   
2  Works great, highly recommend. Great alternati...   

                                          categories  
0  Back To College,College Electronics,College Tv...  
1  Back To College,College Electronics,College Tv...  
2  Back To College,College Electronics,College Tv...  
Attempting to call sonar-pro at https://api.perplexity.ai via the configured SDK...

--- Model Response ---
The current streaming device market is highly competitive, with several standout products offering exceptional performance, ease of use, and value. Based on the available product data and comparison insights

This script loads a CSV file called top_streaming_products.csv, which contains information about different streaming products and their customer reviews. It then identifies the three most positively reviewed products and the one worst-rated product by sorting them according to a "positive_proba" score—an indicator of the likelihood of positive customer feedback.

After collecting product names, scores, categories, and sample reviews, the script constructs a detailed prompt. It uses this prompt to ask a large language model (via the Perplexity API) to write a high-quality, blog-style article comparing the top products, highlighting their differences, main complaints, and why to avoid the worst product. The result is an expert written review article that can help consumers choose the best streaming product for their needs

In [12]:
print(df_imported.columns)

Index(['product_name', 'positive_proba', 'review_text', 'categories'], dtype='object')
